In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt

In [54]:
df = pd.read_csv(r'amazon_baby.csv')

import unicodedata
import sys
tbl = dict.fromkeys(i for i in range(sys.maxunicode)
                      if unicodedata.category(chr(i)).startswith('P'))
def remove_punctuation(text):
    return text.translate(tbl)

# Some data cleaning steps
df['review'] = df['review'].astype(str)
df['review'] = df['review'].apply(remove_punctuation)
df = df.fillna({'review':''})
df = df[df['rating'] != 3]
df['sentiment'] = df['rating'].apply(lambda rating: +1 if rating > 3 else -1)

# Loading train and test indices
import json
temp = open('module-2-assignment-train-idx.json',)
train_idx = json.load(temp)

temp = open('module-2-assignment-test-idx.json',)
test_idx = json.load(temp)

# defining the train and test data
train_data = df.iloc[train_idx]
test_data = df.iloc[test_idx]

# Checking whether the shapes seem logical
display(test_data)


,name,review,rating,sentiment
9,"Baby Tracker&reg; - Daily Childcare Journal, S...",This has been an easy way for my nanny to reco...,4,1
10,"Baby Tracker&reg; - Daily Childcare Journal, S...",I love this journal and our nanny uses it ever...,4,1
16,Nature's Lullabies First Year Sticker Calendar,I love this little calender you can keep track...,5,1
20,Nature's Lullabies Second Year Sticker Calendar,I had a hard time finding a second year calend...,5,1
28,"Lamaze Peekaboo, I Love You",One of babys first and favorite books and it i...,4,1
...,...,...,...,...
183507,Maxboost iPhone 5S/5 Case - Protective Snap-on...,got this for my wife and she loves it would de...,5,1
183515,Maxboost iPhone 5S/5 Case - Protective Snap-on...,I love this phone case My iPhone is always s...,5,1
183522,Airline Seat Belt Extender - The Best Extensio...,I bought this as a fathers day gift for my dad...,5,1
183524,Squeasy Snacker 6oz Silicone Reusable Food Pou...,I love this product it makes my life easier Wi...,5,1


In [29]:
# create sparse dummy variables for each word
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(token_pattern=r'\b\w+\b')
# token_pattern=r'\b\w+\b' to keep single-letter words
# First, learn vocabulary from the training data and assign columns to words
# Then convert the training data into a sparse matrix
train_matrix = vectorizer.fit_transform(train_data['review'])
# Second, convert the test data into a sparse matrix, using the same word-column mapping
test_matrix = vectorizer.transform(test_data['review'])

display(train_matrix.shape, test_matrix.shape)

(133416, 121628)

(33336, 121628)

In [30]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression().fit(train_matrix, train_data['sentiment'])
coef = clf.coef_
test = coef[coef >= 0]
display(len(test))

#Q1 90248 weights are greater than or equal to 0

90248

In [37]:
sample_test_data = test_data[10:13]

sample_test_matrix = vectorizer.transform(sample_test_data['review'])
scores = clf.decision_function(sample_test_matrix)
predictions = clf.predict(sample_test_matrix)
probabilities = clf.predict_proba(sample_test_matrix)

print(scores, probabilities, predictions)

# Q2: the third one

[  5.30701981  -3.23348745 -10.97641989] [[4.93222909e-03 9.95067771e-01]
 [9.62075203e-01 3.79247970e-02]
 [9.99982900e-01 1.70999164e-05]] [ 1 -1 -1]


In [53]:
probabilities = pd.DataFrame(clf.predict_proba(test_matrix))
probabilities_pos = probabilities.sort_values(1, ascending=[False])
pos_idxs = list(probabilities_pos[0:20].index.values)
#display(pos_idxs)
#display(test_data.iloc[pos_idxs])

# Q3 + Q4 kijk in antwoorden

In [56]:
accuracy = clf.score(test_matrix, test_data['sentiment'])
print(accuracy)
# Q5 0.93 accuracy

0.931695464362851


In [79]:
significant_words = ['love', 'great', 'easy', 'old', 'little', 'perfect', 'loves', 
      'well', 'able', 'car', 'broke', 'less', 'even', 'waste', 'disappointed', 
      'work', 'product', 'money', 'would', 'return']

#vectorizer_word_subset = CountVectorizer(vocabulary=significant_words) # limit to 20 words
#train_matrix_word_subset = vectorizer_word_subset.fit_transform(train_data['review'])
#test_matrix_word_subset = vectorizer_word_subset.transform(test_data['review'])

from sklearn.linear_model import LogisticRegression
#simple_model = LogisticRegression().fit(train_matrix_word_subset, train_data['sentiment'])
simple_model_coef = simple_model.coef_
display(simple_model_coef)

# Q7 10 words are positive for the simple model


array([[ 1.36377595,  0.94417756,  1.19164654,  0.08540229,  0.52010571,
         1.51013765,  1.67326465,  0.5035031 ,  0.19107428,  0.05888904,
        -1.65201149, -0.20930074, -0.51151008, -2.03445058, -2.34817862,
        -0.62126407, -0.32084749, -0.89797449, -0.36210025, -2.10966608]])

In [97]:
# Majority class classifies
pos_count = len(test_data[test_data['sentiment'] == 1])
neg_count = len(test_data[test_data['sentiment'] == -1])
accuracy_pos = pos_count/(pos_count + neg_count)
accuracy_neg = neg_count/(pos_count + neg_count)
display(accuracy_pos)
display(accuracy_neg)

0.8427825773938085

0.1572174226061915